## Segmenting and Clustering Neighborhoods in Toronto

In [6]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [8]:

import requests
from pandas.io.json import json_normalize

In [9]:
import folium

In [12]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0

In [13]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [14]:
CLIENT_ID = 'OT1XN4STDNQCNEBFQ4AKE25Z3JBAWWVCI0JXPOFKCS1MTBVH' # your Foursquare ID
CLIENT_SECRET = 'ZY41DW24OLBGGCGZAXZSOIU54DDLGQQJEGFEEDX2EPZMAYJ3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OT1XN4STDNQCNEBFQ4AKE25Z3JBAWWVCI0JXPOFKCS1MTBVH
CLIENT_SECRET:ZY41DW24OLBGGCGZAXZSOIU54DDLGQQJEGFEEDX2EPZMAYJ3


In [15]:

source = urllib.request.urlopen(' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(source, 'html.parser')
tables = soup.find_all('table', class_='sortable')

df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    
for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postal_code, borough, neighborhood = [td.text.strip() for td in tds[:3]]
        df = df.append(
            {'PostalCode': postal_code, 
             'Borough': borough, 
             'Neighborhood': neighborhood
            }, ignore_index=True
        )

In [16]:
df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [17]:
neighborhood_data = df[df['Borough'] != 'Not assigned']
neighborhood_data.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
neighborhood_data.shape

(103, 3)

## Part 2


In [19]:
!wget -q -O 'geographical_coordinates.json' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [20]:
longitude_latitude = pd.read_csv('geographical_coordinates.json')
longitude_latitude.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
longitude_latitude.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:

print(neighborhood_data.shape)
print(longitude_latitude.shape)

(103, 3)
(103, 3)


In [22]:

neighborhood_location = neighborhood_data.join(longitude_latitude, rsuffix='_')
neighborhood_location.drop('PostalCode_', axis=1, inplace=True)
neighborhood_location.dropna(axis=0, inplace=True)
neighborhood_location.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.744734,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029


## Part 3

In [23]:
toronto = neighborhood_location[neighborhood_location['Borough'].apply(lambda x: 'Toronto' in x)]
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.781638,-79.304302
22,M5C,Downtown Toronto,St. James Town,43.770120,-79.408493
30,M4E,East Toronto,The Beaches,43.737473,-79.464763



We have geolocation data on the level of postal code, which is why the combination of postal code and borough will be chosen for the analysis as opposed to neighborhoods in the New York setting.

First we obtain the geolocation information of the city of Toronto.

In [24]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Let us begin by visualizing the information that we have obtained so far.

In [25]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postal_code in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['PostalCode']):
    label = '{}, {}'.format(postal_code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [26]:
toronto.reset_index().at[0, 'PostalCode']

'M5A'

In [27]:
postal_code_latitude = toronto.reset_index().loc[0, 'Latitude'] # neighborhood latitude value
postal_code_longitude = toronto.reset_index().loc[0, 'Longitude'] # neighborhood longitude value

postal_code_name = toronto.reset_index().loc[0, 'PostalCode'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(
    postal_code_name, 
    postal_code_latitude, 
    postal_code_longitude
))


Latitude and longitude values of M5A are 43.773136, -79.23947609999999.


In [28]:
LIMIT = 100  # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    postal_code_latitude, 
    postal_code_longitude, 
    radius, 
    LIMIT)

In [29]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f374cffe70bb116fe60f6bc'},
 'response': {'headerLocation': 'Woburn',
  'headerFullLocation': 'Woburn, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.7776360045, 'lng': -79.2332557734104},
   'sw': {'lat': 43.7686359955, 'lng': -79.24569642658957}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e261f261f6eb1ae13930699',
       'name': "Drupati's Roti & Doubles",
       'location': {'address': '1085 Bellamy Rd N',
        'crossStreet': 'Bellamy & Ellesmere',
        'lat': 43.775222138791534,
        'lng': -79.24167761001029,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775222138791534,
          'lng': -79.24167761001029}],
        'dista

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
1,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
2,Thai One On,Thai Restaurant,43.774468,-79.241268
3,TD Canada Trust,Bank,43.774830,-79.241251
4,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


## Explore all postal codes

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
toronto_venues = getNearbyVenues(names=toronto['PostalCode'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P


In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(523, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
1,M5A,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
2,M5A,43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant
3,M5A,43.773136,-79.239476,TD Canada Trust,43.774830,-79.241251,Bank
4,M5A,43.773136,-79.239476,Centennial Recreation Centre,43.774593,-79.236500,Athletics & Sports


In [36]:
toronto_venues[['PostalCode', 'Venue']].groupby('PostalCode').count()

,Venue
PostalCode,
M4E,2
M4K,36
M4L,17
M4M,35
M4P,4
M5A,8
M5B,14
M5C,34
M5E,6


In [37]:

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 173 uniques categories.


## Analyze each postal codes

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
toronto_onehot.shape

(523, 174)

In [40]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped


,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.027778,0.00,0.00000,0.00,0.000000,0.027778,0.000000,0.000000,0.00,0.027778
2,M4L,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.028571,0.000000,0.000000,0.000000,0.00,0.028571
4,M4P,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M5A,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M5B,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,M5C,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.029412,0.000000,0.00,0.000000
8,M5E,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M5G,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00,0.00,...,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [41]:
num_top_venues = 5

for hood in toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                venue  freq
0             Airport   0.5
1                Park   0.5
2       Movie Theater   0.0
3  Light Rail Station   0.0
4        Liquor Store   0.0


----M4K----
                 venue  freq
0                 Café  0.14
1            Bookstore  0.06
2  Japanese Restaurant  0.06
3       Sandwich Place  0.06
4                  Bar  0.06


----M4L----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3     Baby Store  0.06
4      Nightclub  0.06


----M4M----
              venue  freq
0              Café  0.09
1  Sushi Restaurant  0.06
2       Coffee Shop  0.06
3               Pub  0.06
4       Pizza Place  0.06


----M4P----
                 venue  freq
0  Rental Car Location  0.25
1            Drugstore  0.25
2                  Bar  0.25
3        Garden Center  0.25
4              Airport  0.00


----M5A----
                  venue  freq
0      Hakka Restaurant  0.12
1                Bakery  0.12
2           Gas St

In [42]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_code_venues_sorted = pd.DataFrame(columns=columns)
postal_code_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    postal_code_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postal_code_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Airport,Park,General Travel,Drugstore,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
1,M4K,Café,Sandwich Place,Bar,Restaurant,Japanese Restaurant,Bakery,Bookstore,Dessert Shop,Beer Bar,Beer Store
2,M4L,Grocery Store,Café,Park,Italian Restaurant,Athletics & Sports,Nightclub,Candy Store,Restaurant,Diner,Baby Store
3,M4M,Café,Sushi Restaurant,Pub,Pizza Place,Italian Restaurant,Coffee Shop,Fish & Chips Shop,Latin American Restaurant,Indie Movie Theater,Health Food Store
4,M4P,Drugstore,Garden Center,Bar,Rental Car Location,Dance Studio,Donut Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop


## Cluster postal codes

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 2, 1, 1, 1, 4, 1], dtype=int32)